In [21]:
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import warnings
import lightgbm as lgb
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")
from h2o.automl import H2OAutoML
import h2o

In [23]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,32 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,"28 days, 19 hours and 7 minutes"
H2O_cluster_name:,H2O_from_python_UsuÃ¡rio_q70hek
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.752 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [2]:
ativo = 'wdo_1'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'{ativo}.csv', names = colnames, skiprows=1)

In [3]:
df['MA_5'] = ta.SMA(df['Close'],timeperiod=5)
df['MA_21'] = ta.SMA(df['Close'],timeperiod=21)
df['SAR'] = ta.SAR(df['High'], df['Low'], acceleration=0, maximum=0)
df['BB_up'],df['BB_mid'],df['BB_low'] = ta.BBANDS(df['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
df['OBV'] = ta.OBV(df['Close'],df['Vol'])
df['macd'], df['macdsignal'], df['macdhist'] = ta.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
df['fastk'], df['fastd'] = ta.STOCHF(df['High'], df['Low'], df['Close'], fastk_period=5, fastd_period=3, fastd_matype=0)
df['RSI'] = ta.RSI(df['Close'],14)
df['MOM'] = ta.MOM(df['Close'], timeperiod=10)
df['TRIX'] = ta.TRIX(df['Close'], timeperiod=30)
df['CCI'] = ta.CCI(df['High'], df['Low'], df['Close'], timeperiod=14)
df['ADX'] = ta.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)


tempo_fechar = -10
df['MA_9'] = ta.EMA(df['Close'], timeperiod = 9)
df['MA_9_diff_value'] = df['MA_9'].diff(periods = 1)
df['MA_9_tend'] = df['MA_9_diff_value'].apply(lambda x: 1 if x > 0 else -1)
df['MA_9_tend_diff'] = df['MA_9_tend'].diff()

df['acao'] = df['MA_9_tend_diff'].apply(lambda x: 'call' if x == 2 else 'sell' if  x ==-2 else 0)
df['preco'] = df.apply(lambda x: x['High'] if x['acao']=='call' else x['Low'] if x['acao'] == 'sell' else 0, axis =1)
df['stop'] = df.apply(lambda x: x['Low'] if x['acao']=='call' else x['High'] if x['acao'] == 'sell' else 0, axis =1)

df['realizacao_simplista'] = df['Close'].shift(tempo_fechar)

df = df.dropna()


df_acao = df[df['acao'] != 0]



def cal_result(x):
    if x['acao'] == 'call':
        if x['realizacao_simplista'] < x['stop']:
            resultado = x['stop'] - x['preco']
            return resultado
        else:
            resultado = x['realizacao_simplista'] - x['preco']
            return resultado
    if x['acao'] == 'sell':
        if x['realizacao_simplista'] > x['stop']:
            resultado = x['stop'] - x['preco']
            return resultado
        else:
            resultado = x['realizacao_simplista'] - x['preco']
            return resultado

df_acao['resultado_valor'] = df_acao.apply(cal_result, axis = 1)
df_acao['resultado_binario'] = df_acao['resultado_valor'].apply(lambda x: 1 if x > 0 else 0)


compra = df_acao[df_acao['acao'] =='call']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='call'].shape[0]
venda = df_acao[df_acao['acao'] =='sell']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='sell'].shape[0]

compra_result = df_acao[df_acao['acao'] =='call']['resultado_valor'].sum()
venda_result = df_acao[df_acao['acao'] =='sell']['resultado_valor'].sum()

print(f'Venda: {venda}% -- Compra: {compra}%')
print(f'Venda: {venda_result} -- Compra: {compra_result}')

Venda: 0.5584415584415584% -- Compra: 0.42502951593860683%
Venda: -232.0 -- Compra: 649.0


In [14]:
df_new = df_acao.drop(columns = ['MA_9_diff_value', 'MA_9_tend', 'MA_9_tend_diff', 'preco','stop', 'realizacao_simplista'])
df_new = df_new.set_index(['Data','resultado_valor'])

In [267]:
df_new_sell = df_new[df_new['acao'] == 'sell']
df_new_sell = df_new_sell.drop(columns = ['acao'])

In [15]:
df_new_sell = df_new.drop(columns = ['acao'])

In [25]:
df_new_sell.to_csv('df_h2o.csv', index = False)

In [26]:
df = h2o.import_file('df_h2o.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [37]:
train, valid, test = df.split_frame(ratios=[0.6,0.2], seed=1234)

response = "resultado_binario"
train[response] = train[response].asfactor()
valid[response] = valid[response].asfactor()
test[response] = test[response].asfactor()

In [34]:
x = train.drop('resultado_binario').columns
y = 'resultado_binario'

In [38]:
aml = H2OAutoML(max_models = 10, max_runtime_secs=100, seed = 1)
aml.train(x=x, y=y, training_frame=train, validation_frame=valid)

AutoML progress: |█
09:12:16.360: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
09:12:16.366: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: XRT_1_AutoML_2_20221222_91216


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    32                 32                          83687                  17           20           19.25         182           224           203.5

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.23032462937469794
RMSE: 0.47992148251010597
LogLoss: 0.8770797863895824
Mean Per-Class Error: 0.38096662741994636
AUC: 0.681505143893918
AUCPR: 0.6595572412959985
Gini: 0.36301028778783606

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3454545627940785
       0    1    Error    Rate
-----  ---  ---  -------  --------------
0      193  310  0.6163   (310.0/503.0)
1      75   440  0.1456   (75.0/515.0)
Total  268  750  0.3782   (385.0/1018.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.345455     0.695652  187
max f2                       0            0.836582  253
max f0point5                 0.461538     0.641026  150
max accuracy                 0.461538     0.638507  150
max precision                0.857143     0.77193   14
max recall                   0            1         253
max specificity              1            0.992048  0
max absolute_mcc             0.389113     0.285615  172
max min_per_class_accuracy   0.503759     0.598058  137
max mean_per_class_accuracy  0.461538     0.637669  150
max tns                      1            499       0
max fns                      1            508       0
max fps                      0            503       253
max tps                      0            515       253
max tnr                      1            0.992048  0
max fnr                      1            0.986408  0
max fpr                      0            1         253
max tpr                      0            1         253

Gains/Lift Table: Avg response rate: 50,59 %, avg score: 50,37 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0108055                   0.993041           1.2579    1.2579             0.636364         1         0.636364                    1                   0.0135922       0.0135922                  25.7899   25.7899            0.00563995
2        0.0255403                   0.916667           1.58136   1.44451            0.8              0.929176  0.730769                    0.95914             0.023301        0.0368932                  58.1359   44.4511            0.0229767
3        0.0324165                   0.9                1.12954   1.3777             0.571429         0.903896  0.69697                     0.947422            0.00776699      0.0446602                  12.9542   37.7699            0.0247795
4        0.040275                    0.875191           1.72961   1.44637            0.875            0.885566  0.731707                    0.935352            0.0135922       0.0582524                  72.9612   44.6365            0.0363836
5        0.0559921                   0.857143           1.72961   1.52587            0.875            0.864732  0.77193                     0.915529            0.027184

In [39]:
lb = aml.leaderboard
lb

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
XRT_1_AutoML_2_20221222_91216,0.707811,0.629922,0.689763,0.371212,0.467428,0.218489
StackedEnsemble_BestOfFamily_1_AutoML_2_20221222_91216,0.706985,0.62513,0.690317,0.393752,0.466384,0.217514
StackedEnsemble_AllModels_1_AutoML_2_20221222_91216,0.703387,0.627288,0.685828,0.395949,0.467495,0.218552
DRF_1_AutoML_2_20221222_91216,0.702683,0.628185,0.696241,0.375396,0.468157,0.219171
GBM_4_AutoML_2_20221222_91216,0.68071,0.641846,0.652828,0.42614,0.474862,0.225494
GBM_grid_1_AutoML_2_20221222_91216_model_1,0.6732,0.648924,0.656427,0.468052,0.477709,0.228205
GBM_2_AutoML_2_20221222_91216,0.672731,0.648223,0.652275,0.427735,0.47754,0.228044
GBM_5_AutoML_2_20221222_91216,0.670681,0.648915,0.658564,0.404524,0.478088,0.228568
GBM_3_AutoML_2_20221222_91216,0.670667,0.646715,0.654668,0.406605,0.477285,0.227801
GBM_1_AutoML_2_20221222_91216,0.614874,0.672211,0.600481,0.480027,0.489651,0.239758


In [40]:
aml.leader

Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: XRT_1_AutoML_2_20221222_91216


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    32                 32                          83687                  17           20           19.25         182           224           203.5

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.23032462937469794
RMSE: 0.47992148251010597
LogLoss: 0.8770797863895824
Mean Per-Class Error: 0.38096662741994636
AUC: 0.681505143893918
AUCPR: 0.6595572412959985
Gini: 0.36301028778783606

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3454545627940785
       0    1    Error    Rate
-----  ---  ---  -------  --------------
0      193  310  0.6163   (310.0/503.0)
1      75   440  0.1456   (75.0/515.0)
Total  268  750  0.3782   (385.0/1018.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.345455     0.695652  187
max f2                       0            0.836582  253
max f0point5                 0.461538     0.641026  150
max accuracy                 0.461538     0.638507  150
max precision                0.857143     0.77193   14
max recall                   0            1         253
max specificity              1            0.992048  0
max absolute_mcc             0.389113     0.285615  172
max min_per_class_accuracy   0.503759     0.598058  137
max mean_per_class_accuracy  0.461538     0.637669  150
max tns                      1            499       0
max fns                      1            508       0
max fps                      0            503       253
max tps                      0            515       253
max tnr                      1            0.992048  0
max fnr                      1            0.986408  0
max fpr                      0            1         253
max tpr                      0            1         253

Gains/Lift Table: Avg response rate: 50,59 %, avg score: 50,37 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0108055                   0.993041           1.2579    1.2579             0.636364         1         0.636364                    1                   0.0135922       0.0135922                  25.7899   25.7899            0.00563995
2        0.0255403                   0.916667           1.58136   1.44451            0.8              0.929176  0.730769                    0.95914             0.023301        0.0368932                  58.1359   44.4511            0.0229767
3        0.0324165                   0.9                1.12954   1.3777             0.571429         0.903896  0.69697                     0.947422            0.00776699      0.0446602                  12.9542   37.7699            0.0247795
4        0.040275                    0.875191           1.72961   1.44637            0.875            0.885566  0.731707                    0.935352            0.0135922       0.0582524                  72.9612   44.6365            0.0363836
5        0.0559921                   0.857143           1.72961   1.52587            0.875            0.864732  0.77193                     0.915529            0.027184

In [41]:
# Get model ids for all models in the AutoML Leaderboard
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
# Get the "All Models" Stacked Ensemble model
se = h2o.get_model([mid for mid in model_ids if "StackedEnsemble_AllModels" in mid][0])

In [42]:
se

Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_1_AutoML_2_20221222_91216

No summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.04161703839024864
RMSE: 0.20400254505826304
LogLoss: 0.21382394018122586
AUC: 1.0
AUCPR: 1.0
Gini: 1.0
Null degrees of freedom: 1017
Residual degrees of freedom: 1011
Null deviance: 1411.1062025129272
Residual deviance: 435.34554220897587
AIC: 449.34554220897587

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5959861714007412
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      503  0    0        (0.0/503.0)
1      0    515  0        (0.0/515.0)
Total  503  515  0        (0.0/1018.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.595986     1         191
max f2                       0.595986     1         191
max f0point5                 0.595986     1         191
max accuracy                 0.595986     1         191
max precision                0.999796     1         0
max recall                   0.595986     1         191
max specificity              0.999796     1         0
max absolute_mcc             0.595986     1         191
max min_per_class_accuracy   0.595986     1         191
max mean_per_class_accuracy  0.595986     1         191
max tns                      0.999796     503       0
max fns                      0.999796     508       0
max fps                      0.000293113  503       399
max tps                      0.595986     515       191
max tnr                      0.999796     1         0
max fnr                      0.999796     0.986408  0
max fpr                      0.000293113  1         399
max tpr                      0.595986     1         191

Gains/Lift Table: Avg response rate: 50,59 %, avg score: 50,62 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0108055                   0.954544           1.9767    1.9767             1                0.983738   1                           0.983738            0.0213592       0.0213592                  97.6699   97.6699            0.0213592
2        0.0206287                   0.932371           1.9767    1.9767             1                0.941153   1                           0.963459            0.0194175       0.0407767                  97.6699   97.6699            0.0407767
3        0.0304519                   0.923024           1.9767    1.9767             1                0.926287   1                           0.951468            0.0194175       0.0601942                  97.6699   97.6699            0.0601942
4        0.040275                    0.919046           1.9767    1.9767             1                0.920909   1                           0.944014            0.0194175       0.0796117                  97.6699   97.6699            0.0796117
5        0.0500982                   0.906916           1.9767    1.9767             1                0.910694   1                           0.937481            0.0194175       0.0990291                  97.6699   97.6699            0.0990291
6        0.100196                    0.875531           1.9767    1.9767             1                0.891931   1                           0.914706            0.0990291       0.198058                   97.6699   97.6699            0.198058
7        0.150295             

In [44]:
a = se.predict(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [46]:
test['t'] = a['predict']

In [47]:
test

Open,High,Low,Close,Vol,MA_5,MA_21,SAR,BB_up,BB_mid,BB_low,OBV,macd,macdsignal,macdhist,fastk,fastd,RSI,MOM,TRIX,CCI,ADX,MA_9,resultado_binario,t
5214.5,5217.5,5213,5214.5,5865,5216.3,5213.05,5246.5,5219.89,5212.73,5205.56,-66223,-2.05,-3.27497,1.22497,28.5714,47.5155,46.1029,1.5,-0.00688383,58.2503,17.3568,5214.55,0,1
5208,5209.5,5203,5204,11081,5204.1,5203,5246.5,5210.55,5203.25,5195.95,-129353,-0.808716,-1.50371,0.694992,50,78.3333,48.2076,-3.5,-0.00801651,30.5556,13.7569,5204.59,1,1
5226.5,5236.5,5226.5,5236,16221,5227.8,5226.98,5246.5,5233.05,5226.93,5220.8,-30729,3.0409,3.18587,-0.144976,96,56.2424,67.9925,11,0.00482041,204.399,33.75,5228.18,0,1
5229,5229,5225,5226,6937,5225.6,5230.5,5246.5,5242.67,5230.7,5218.73,-37022,-0.151572,1.09569,-1.24726,50,55.5556,46.2875,-11.5,0.00647521,-51.8519,22.3757,5227.33,0,0
5216,5221,5214.5,5220.5,7320,5216.5,5223.95,5246.5,5235.73,5223.27,5210.82,-95140,-3.06552,-2.28296,-0.782556,95,67.2817,46.0921,-8,0.00356824,-28.4722,22.9433,5218.7,0,1
5220,5220,5218.5,5219,1747,5219.6,5221.71,5246.5,5230.62,5221.43,5212.23,-98225,-2.36523,-2.37606,0.0108339,60,79.2593,44.0448,-2,0.00253874,-6.94845,19.0772,5219.39,0,1
5216.5,5220,5215,5219,4111,5219.4,5220.81,5246.5,5229.12,5220.52,5211.93,-98757,-2.27679,-2.36333,0.0865366,57.1429,50.1587,45.0501,5,0.00189733,-4.62046,18.4796,5218.93,0,1
5219,5219.5,5216.5,5217,1990,5218.4,5220.36,5246.5,5228.31,5220.05,5211.79,-100747,-2.29775,-2.35021,0.0524654,28.5714,39.6825,42.0283,3.5,0.00158979,0.821596,18.0505,5218.54,0,1
5218.5,5222,5218.5,5220.5,3255,5220.2,5216.93,5246.5,5222.57,5216.98,5211.38,-86529,0.275794,-0.440843,0.716637,72.7273,64.5688,54.6716,5,-0.00204807,84.4961,12.6748,5219.27,0,0
5220,5220,5216.5,5216.5,3939,5218.3,5214.64,5246.5,5220.62,5214.4,5208.18,-77701,0.434037,-0.336138,0.770176,8.33333,55.5556,51.026,7.5,-0.00245894,60.4891,15.1799,5216.94,1,1
